# Cocoa Disease Detection - Optimized YOLOv8 Training

**Dataset:** Roboflow Cocoa Disease Detection v3
- **Total Images:** 3009 (Train: 2099, Valid: 594, Test: 316)
- **Classes:** 4 (BLACKPOD, FROSTYPOD, HEALTHY, MIRID)
- **Preprocessing:** Auto-Orient, Resize to 640x640
- **Roboflow Augmentations:** None (we add augmentations during training)

**Model:** YOLOv8 Small (better accuracy than Nano for this dataset size)

**Training Optimizations:**
1. ✅ **Data Augmentation** - Rotation, scaling, flipping, mosaic, mixup (since Roboflow has none)
2. ✅ **150 Epochs** - More training for robust learning with 3009 images
3. ✅ **Early Stopping** - Patience of 30 epochs to prevent overfitting
4. ✅ **Transfer Learning** - Starting from pretrained YOLOv8 weights
5. ✅ **AdamW Optimizer** - Better convergence than Adam
6. ✅ **Cosine Learning Rate** - Smoother convergence

**Estimated Training Time:**
- CPU: ~4-6 hours
- GPU (T4): ~45-60 minutes
- GPU (A100): ~20-30 minutes

**Recommendation:** Use Google Colab with free T4 GPU for faster training!

In [ ]:
# Install required packages
!pip install roboflow ultralytics opencv-python-headless matplotlib

In [ ]:
# Download dataset from Roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="4T1HDjxUFC63SMVVXxCe")
project = rf.workspace("miss-nyarko-s2gtm").project("cocoa-disease-detection")
version = project.version(3)
dataset = version.download("yolov8")

print(f"Dataset downloaded to: {dataset.location}")

In [ ]:
# Verify dataset structure
import os

dataset_path = dataset.location
print("Dataset structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")

In [ ]:
# Check data.yaml file
import yaml

data_yaml_path = os.path.join(dataset_path, 'data.yaml')
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("Dataset configuration:")
print(f"Number of classes: {data_config.get('nc', 'N/A')}")
print(f"Class names: {data_config.get('names', 'N/A')}")
print(f"Train path: {data_config.get('train', 'N/A')}")
print(f"Val path: {data_config.get('val', 'N/A')}")

In [ ]:
# Initialize YOLOv8 nano model
from ultralytics import YOLO

# Start with pretrained YOLOv8 nano (fastest for real-time detection)
model = YOLO('yolov8s.pt')

print("YOLOv8 nano model loaded successfully!")

In [ ]:
# Train the model with optimized settings for GPU training
results = model.train(
    data=data_yaml_path,
    epochs=150,              # More epochs for robust training
    imgsz=640,               # Image size
    batch=32,                # Larger batch size for GPU (was 16 for CPU)
    device='cuda',           # Use GPU (T4)
    patience=30,             # Early stopping patience
    save=True,               # Save checkpoints
    project='runs/detect',   # Project directory
    name='cocoa_disease',    # Experiment name
    exist_ok=True,           # Overwrite if exists
    pretrained=True,         # Use pretrained weights
    optimizer='AdamW',       # AdamW optimizer
    verbose=True,            # Verbose output
    seed=42,                 # Random seed for reproducibility
    deterministic=True,      # Deterministic training
    single_cls=False,        # Multi-class detection (4 classes)
    rect=False,              # Rectangular training
    cos_lr=True,             # Cosine learning rate scheduler
    close_mosaic=15,         # Close mosaic in last 15 epochs
    resume=False,            # Resume training
    amp=True,                # Automatic Mixed Precision - FASTER on GPU!
    fraction=1.0,            # Use 100% of dataset
    profile=False,           # Profile ONNX runtime
    freeze=None,             # Don't freeze layers
    lr0=0.001,               # Initial learning rate
    lrf=0.01,                # Final learning rate
    momentum=0.937,          # SGD momentum
    weight_decay=0.0005,     # Weight decay
    warmup_epochs=5.0,       # Warmup epochs
    warmup_momentum=0.8,     # Warmup momentum
    warmup_bias_lr=0.1,      # Warmup bias learning rate
    box=7.5,                 # Box loss gain
    cls=0.5,                 # Class loss gain
    dfl=1.5,                 # DFL loss gain
    
    # Data Augmentation (CRITICAL - Roboflow has NO augmentations!)
    hsv_h=0.015,             # HSV-Hue augmentation
    hsv_s=0.7,               # HSV-Saturation augmentation
    hsv_v=0.4,               # HSV-Value augmentation
    degrees=15.0,            # Rotation augmentation ±15°
    translate=0.2,           # Translation augmentation (20%)
    scale=0.6,               # Scale augmentation (40-140%)
    shear=5.0,               # Shear augmentation ±5°
    perspective=0.0003,      # Perspective augmentation
    flipud=0.0,              # No vertical flip
    fliplr=0.5,              # 50% horizontal flip
    mosaic=1.0,              # Mosaic augmentation (very effective!)
    mixup=0.15,              # Mixup augmentation
    copy_paste=0.1,          # Copy-paste augmentation
    
    # Advanced settings
    dropout=0.0,             # No dropout
    val=True,                # Validate during training
    plots=True,              # Save training plots
    save_period=-1,          # Only save best/last
)

print("\n" + "="*50)
print("Training completed!")
print("="*50)
print(f"\nDataset stats:")
print(f"- Total images: 3009")
print(f"- Training images: 2099")
print(f"- Validation images: 594")
print(f"- Test images: 316")
print(f"- Classes: 4 (BLACKPOD, FROSTYPOD, HEALTHY, MIRID)")
print(f"\nOptimizations:")
print(f"- Model: YOLOv8 Small")
print(f"- Device: GPU (CUDA)")
print(f"- Batch size: 32 (optimized for T4 GPU)")
print(f"- Mixed Precision: Enabled (AMP)")
print(f"- Augmentations: Rotation, Scale, Flip, Mosaic, Mixup, Copy-Paste")
print(f"- Epochs: 150 (with early stopping after 30 epochs no improvement)")

In [ ]:
# Evaluate the model
metrics = model.val()

print("\nValidation Metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

In [ ]:
# Copy best model to project directory with verification
import shutil
from ultralytics import YOLO

# Path to best trained model
best_model_path = 'runs/detect/cocoa_disease/weights/best.pt'
destination_path = 'public/models/best.pt'

# Check if source file exists
if not os.path.exists(best_model_path):
    print(f"❌ ERROR: Model not found at {best_model_path}")
    print("Training may have failed or saved to a different location.")
    print("\nSearching for best.pt...")
    for root, dirs, files in os.walk('runs'):
        for file in files:
            if file == 'best.pt':
                print(f"Found: {os.path.join(root, file)}")
else:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    
    # Copy the model
    shutil.copy2(best_model_path, destination_path)
    
    # Verify the copy
    if os.path.exists(destination_path):
        file_size = os.path.getsize(destination_path)
        print(f"✅ Model successfully saved to: {destination_path}")
        print(f"📦 Model size: {file_size / (1024*1024):.2f} MB")
        
        # Load and verify the model
        try:
            model = YOLO(destination_path)
            print(f"🎯 Classes detected: {model.names}")
            print(f"📊 Number of classes: {len(model.names)}")
            
            # Download option for Google Colab
            try:
                from google.colab import files
                print("\n📥 Download the model to your computer:")
                print("Run this to download:")
                print(f"files.download('{destination_path}')")
            except:
                print("\n💻 Not running in Colab - model saved locally")
                
        except Exception as e:
            print(f"⚠️ Warning: Could not verify model: {e}")
    else:
        print(f"❌ ERROR: Failed to copy model to {destination_path}")

In [ ]:
# Test the model with a sample image
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Load the best model
trained_model = YOLO(destination_path)

# Get a test image from the dataset
test_images_dir = os.path.join(dataset_path, 'test', 'images')
if not os.path.exists(test_images_dir):
    test_images_dir = os.path.join(dataset_path, 'valid', 'images')

test_image_files = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
test_image_path = os.path.join(test_images_dir, test_image_files[0])

# Run inference
results = trained_model(test_image_path, conf=0.25)

# Display results
print(f"\nTest image: {test_image_files[0]}")
print(f"Detections: {len(results[0].boxes)}")

# Show the image with detections
annotated_image = results[0].plot()
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Cocoa Disease Detection Result')
plt.show()

# Print detection details
if len(results[0].boxes) > 0:
    for i, box in enumerate(results[0].boxes):
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        class_name = trained_model.names[class_id]
        print(f"Detection {i+1}: {class_name} ({confidence:.2%})")
else:
    print("No detections found")

In [ ]:
# Display training results
from IPython.display import Image as IPImage, display

results_dir = 'runs/detect/cocoa_disease'

print("Training Results:")
print("\n1. Confusion Matrix:")
display(IPImage(filename=f'{results_dir}/confusion_matrix.png'))

print("\n2. Training Curves:")
display(IPImage(filename=f'{results_dir}/results.png'))

print("\n3. Sample Predictions:")
val_batch_path = f'{results_dir}/val_batch0_pred.jpg'
if os.path.exists(val_batch_path):
    display(IPImage(filename=val_batch_path))

## Summary

**Model trained successfully!**

**Classes:**
- 0: BLACKPOD
- 1: FROSTYPOD
- 2: HEALTHY
- 3: MIRID

**Next steps:**
1. Model saved to `public/models/best.pt`
2. Update backend API class names
3. Update frontend for new classes
4. Test with live camera detection